# 《手撕AutoGPT》精细拆解

<div class="alert alert-warning">
<b>实例来源：</b><br/>
本文中的实例主要来自AGI课堂培训内容《手撕AutoGPT》，感谢王卓然老师提供了优秀的核心代码实现。<p>
</div>

那为什么还要专门编写一篇文章重复王卓然老师讲过内容呢？

因为智能体的内容比较复杂，本文根据王卓然老师的视频讲解课程和代码示例，进一步拆解为 JupyterLab 上运行，并探索了结合AgentExcuter实现，以便更好融入到 langchain 的体系结构。

## 阅读本文后你可以获得

1. 在 Jupyter 的笔记模式下，从零开始构建一个复杂的智能体
2. 拆开智能体的零部件，单步骤执行，了解智能体的运行机理
3. 将零部件组装好，构建一个智能体类
4. 将自定义的智能体装入到 langchain 中的 AgentExcuter 框架中运行

## 实现思路

### 构建智能体的一般性思路

如果要将大模型融入到你的应用中，最佳的方案是构建一个智能体。

常见的智能体结构都会包含三个核心环节：
- 思考
- 行动
- 观察

其中，图中Planning部份是**思考环节**，这是最重要的环节，也是典型的 LLM+Prompt+OutputParser 的结构。

智能体的LLM必须是推理能力足够强的大模型，例如GPT4，否则无法完成任务。<br>
智能体的Prompt一般依照相关论文的思路拟定思考逻辑，例如 ReAct 模式，下面的实践中还会包括反思、思维链等技巧。<br>
智能体需要调度哪个Action、需要什么输入参数，都由OutputParser负责解析。

如果你希望结合langchain的AgentExcuter，以便获得类似LCEL的福利，还必须提供特定的Prompt字段和输出解析结构。<br>
后面的实践会详细解释。

图中Action部份就是**行动环节**，就是根据思考结果（或称为计划）去调用工具，这个过程在AgentExcuter中有自己的实现逻辑，如果不使用他就要自己实现。

图中的Tools是部份**行动环节**可调用的外部工具，langchain 内置了一部份开箱即用的工具，但更多时候需要自己实现。

**观察环节**就是将工具调用的结果装入**思考环节**的提示语中，以便大模型在推理时使用。

图中的Memory是记忆部份，是智能体每个步骤或每次任务的执行结果，对应步骤的结果称为短期记忆，对应任务的执行结果称为长期记忆。

经过这些环节，智能体的运行就会形成一个完整闭环：思考、行动、观察，再思考、行动、观察 ... 经过N次循环后得出结论，这就是智能体的核心实现逻辑。

![](./images/agent-overview.png)

### 本文中智能体的实现逻辑

本文的实现实现逻辑是典型的ReAct策略，但其中增加了一些思维链技巧：

1. 用户输入
2. 思考：调用LLM思考
3. 判断：根据思考结果，判断是已经获得答案，还是需要调用工具
4. 行动：如果调用工具，就在工具箱中执行
5. 观察：将执行结果装入短时记忆
6. 回到第2步骤继续，直到思考结果中包括了FINISH
7. 生成最终答案

![](./images/agent-flowchart.png)

## 准备工作

### 加载环境变量

你必须自己准备好OpenAI的的Key，在本文中需要使用GPT4的模型。

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

### 着色打印工具

王卓然老师的代码中提供个了几个有趣的小工具，这是其中之一：

In [2]:
from colorama import init, Fore, Back, Style
import sys

THOUGHT_COLOR = Fore.GREEN
OBSERVATION_COLOR = Fore.YELLOW
ROUND_COLOR = Fore.RED
CODE_COLOR = Fore.BLUE

def color_print(text, color=None, end="\n"):
    if color is not None:
        content = color + text + Style.RESET_ALL + end
    else:
        content = text + end
    sys.stdout.write(content)
    sys.stdout.flush()

使用示例：

In [3]:
color_print("我现在开始思考...", color = THOUGHT_COLOR)
color_print("第一轮开始了", color = ROUND_COLOR)
color_print("def hello(): \n    print('hi')", color = CODE_COLOR)

我现在开始思考...
第一轮开始了
def hello(): 
    print('hi')


### 中文转换工具

后面在将Pandtic类型转换为提示语的一部份时会用到，没有这部份会输出Unicode编码，很不友好。

In [4]:
def _chinese_friendly(string) -> str:
    lines = string.split('\n')
    for i, line in enumerate(lines):
        if line.startswith('{') and line.endswith('}'):
            try:
                lines[i] = json.dumps(json.loads(line), ensure_ascii=False)
            except:
                pass
    return '\n'.join(lines)

### Tool的包装函数

- 将异常转换为文字
- 将输出结果转换为文本

In [140]:
def _safe_func_call(func, *args, **kwargs):
    """Call a function with any arguments, return error message if an exception is raised"""
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return str(e)

from functools import wraps

def make_safe_tool(func):
    """Create a new function that wraps the given function with error handling"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return str(func(*args, **kwargs))
        except Exception as e:
            return str(e)
    return wrapper

## tools 定义

In [141]:
from langchain.agents import Tool
from langchain.tools import StructuredTool

### 文件处理

#### 列举文件

In [142]:
import os

def list_files_in_directory(path: str) -> str:
    """List all file names in the directory"""
    file_names = os.listdir(path.strip())

    return "\n".join(file_names)

#### 定义 tool 函数

In [143]:
# 输出工具
directory_inspection_tool = StructuredTool.from_function(
    func=make_safe_tool(list_files_in_directory),
    name="ListDirectory",
    description="探查文件夹的内容和结构，展示它的文件名和文件夹名",
)

#### 使用示例

In [144]:
# 参数中的文件目录需要修改为你自己的目录
print(list_files_in_directory("./data/autogpt-demo"))

.DS_Store
2023年8月-9月销售记录.xlsx
供应商名录.xlsx
求职简历.docx
供应商资格要求.pdf


你也可以将这些工具当作 Runnable 对象来使用：

In [10]:
directory_inspection_tool

StructuredTool(name='ListDirectory', description='ListDirectory(path: str) -> str - 探查文件夹的内容和结构，展示它的文件名和文件夹名', args_schema=<class 'pydantic.v1.main.ListDirectorySchemaSchema'>, func=<function list_files_in_directory at 0x116043910>)

In [11]:
list_files_in_directory

<function __main__.list_files_in_directory(path: str) -> str>

In [12]:
make_safe_tool(list_files_in_directory)

<function __main__.list_files_in_directory(path: str) -> str>

In [13]:
print(make_safe_tool(list_files_in_directory)("./data/autogpt-demo"))

.DS_Store
2023年8月-9月销售记录.xlsx
供应商名录.xlsx
求职简历.docx
供应商资格要求.pdf


In [14]:
print(directory_inspection_tool.invoke("./data/autogpt-demo"))

.DS_Store
2023年8月-9月销售记录.xlsx
供应商名录.xlsx
求职简历.docx
供应商资格要求.pdf


cannot create 'generator' instances


### 查询文档

实现一个简单的RAG，以便从PDF文档或word文档中查询文本内容。

<div class="alert alert-info">
<b>加载Word：</b><br/>
因为python环境兼容问题，这里我修改了word文档的加载类。
</div>

#### 准备

In [145]:
from typing import List
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI, OpenAIEmbeddings

In [146]:
def get_file_extension(filename: str) -> str:
    return filename.split(".")[-1]

In [147]:
class FileLoadFactory:
    @staticmethod
    def get_loader(filename: str):
        filename = filename.strip()
        ext = get_file_extension(filename)
        if ext == "pdf":
            return PyPDFLoader(filename)
        elif ext == "docx" or ext == "doc":
            return Docx2txtLoader(filename)
        else:
            raise NotImplementedError(f"File extension {ext} not supported.")

In [148]:
def load_docs(filename: str) -> List[Document]:
    file_loader = FileLoadFactory.get_loader(filename)
    pages = file_loader.load_and_split()
    return pages

#### 使用 RAG 查询文档

In [149]:
def ask_docment(
        filename: str,
        query: str,
) -> str:
    """根据一个PDF文档的内容，回答一个问题"""

    raw_docs = load_docs(filename)
    if len(raw_docs) == 0:
        return "抱歉，文档内容为空"
    text_splitter = RecursiveCharacterTextSplitter(
                        chunk_size=1000,
                        chunk_overlap=200,
                        length_function=len,
                        add_start_index=True,
                    )
    documents = text_splitter.split_documents(raw_docs)
    if documents is None or len(documents) == 0:
        return "无法读取文档内容"
    db = Chroma.from_documents(documents, OpenAIEmbeddings(model="text-embedding-ada-002"))
    qa_chain = RetrievalQA.from_chain_type(
        llm=OpenAI(
            temperature=0,
            model_kwargs={
                "seed": 42
            },
        ),  # 语言模型
        chain_type="stuff",  # prompt的组织方式
        retriever=db.as_retriever()  # 检索器
    )
    response = qa_chain.invoke(query+"(请用中文回答)")
    return response

#### 定义 tool 函数

In [150]:
# 输出工具
document_qa_tool = StructuredTool.from_function(
    func=make_safe_tool(ask_docment),
    name="AskDocument",
    description="根据一个Word或PDF文档的内容，回答一个问题。考虑上下文信息，确保问题对相关概念的定义表述完整。",
)

#### 使用示例

In [153]:
# 使用示例
filename = "./data/autogpt-demo/供应商资格要求.pdf"
query = "销售额的达标标准是多少？"
result = ask_docment(filename, query)
print(type(result).__name__)
print(result)

dict
{'query': '销售额的达标标准是多少？(请用中文回答)', 'result': ' 与我方合作的月份，经我方销售的产品月总价不得低于人民币3万元。'}


<div class="alert alert-warning">
    <b>工具的输出必须是字符串</b><br>
make_safe_tool已经确保工具执行结果是字符串，而不是字典等其他类型。    
</div>

In [155]:
result = document_qa_tool.invoke({"filename": filename, "query": "销售额的达标标准是多少？"})
print(type(result).__name__)
print(result)

str
{'query': '销售额的达标标准是多少？(请用中文回答)', 'result': ' 与我方合作的月份，经我方销售的产品月总价不得低于人民币3万元。'}


In [156]:
# 使用示例
filename = "./data/autogpt-demo/求职简历.docx"
query = "工作经历如何？"
print(ask_docment(filename, query))

{'query': '工作经历如何？(请用中文回答)', 'result': ' 我不知道，我只知道他们的联系电话和电子邮箱。'}


### Excel结构探查

#### 探查Excel的sheet、列名和前N行数据

In [157]:
import pandas as pd

In [158]:
def get_sheet_names(
        filename : str
) -> str :
    """获取 Excel 文件的工作表名称"""
    excel_file = pd.ExcelFile(filename.strip())
    sheet_names = excel_file.sheet_names
    return f"这是 '{filename}' 文件的工作表名称：\n\n{sheet_names}"

In [159]:
def get_column_names(
        filename : str
) -> str:
    """获取 Excel 文件的列名"""

    # 读取 Excel 文件的第一个工作表
    df = pd.read_excel(filename.strip(), sheet_name=0)  # sheet_name=0 表示第一个工作表
    column_names = '\n'.join(
        df.columns.to_list()
    )

    result = f"这是 '{filename.strip()}' 文件第一个工作表的列名：\n\n{column_names}"
    return result

In [160]:
def get_first_n_rows(
        filename : str,
        n : int = 3
) -> str :
    """获取 Excel 文件的前 n 行"""

    filename = filename.strip()
    result = get_sheet_names(filename)+"\n\n"
    result += get_column_names(filename)+"\n\n"

    # 读取 Excel 文件的第一个工作表
    df = pd.read_excel(filename, sheet_name=0)  # sheet_name=0 表示第一个工作表
    n_lines = '\n'.join(
        df.head(n).to_string(index=False, header=True).split('\n')
    )

    result += f"这是 '{filename}' 文件第一个工作表的前{n}行样例：\n\n{n_lines}"
    return result

#### 定义 tool 函数

<div class="alert alert-info">
<b>增加Tool的参数说明：</b><br/>
    实践中，我发现大模型有时会弄错参数，虽然会迭代纠正，但还是习惯性做了优化，在工具说明中补充了参数说明。
</div>

In [161]:
# 输出工具
excel_inspection_tool = StructuredTool.from_function(
    func=make_safe_tool(get_first_n_rows),
    name="InspectExcel",
    description="""
    探查表格文件的内容和结构，展示它的列名和前n行，n默认为3。
    
    使用该函数时应当准备提供filename和n两个参数，其中：
    
    - filename：要探查的Excel文件名
    - n: 默认的行数
    
    """,
)

#### 使用示例

In [336]:
print(get_first_n_rows("./data/autogpt-demo/供应商名录.xlsx"))

这是 './data/autogpt-demo/供应商名录.xlsx' 文件的工作表名称：

['Sheet1']

这是 './data/autogpt-demo/供应商名录.xlsx' 文件第一个工作表的列名：

供应商
联系人
联系人邮箱
内部对接人

这是 './data/autogpt-demo/供应商名录.xlsx' 文件第一个工作表的前3行样例：

       供应商 联系人                   联系人邮箱 内部对接人
  北京科技有限公司  张伟 xiaoming123@example.com    李华
  上海音响有限公司  李芳    lihua456@example.org    张伟
深圳创新科技有限公司  陈瑶   wangli789@example.net    王芳


In [338]:
print(excel_inspection_tool.invoke("./data/autogpt-demo/供应商名录.xlsx"))

这是 './data/autogpt-demo/供应商名录.xlsx' 文件的工作表名称：

['Sheet1']

这是 './data/autogpt-demo/供应商名录.xlsx' 文件第一个工作表的列名：

供应商
联系人
联系人邮箱
内部对接人

这是 './data/autogpt-demo/供应商名录.xlsx' 文件第一个工作表的前3行样例：

       供应商 联系人                   联系人邮箱 内部对接人
  北京科技有限公司  张伟 xiaoming123@example.com    李华
  上海音响有限公司  李芳    lihua456@example.org    张伟
深圳创新科技有限公司  陈瑶   wangli789@example.net    王芳


### Excel数据分析

#### 准备

In [162]:
import re
from langchain.tools import StructuredTool
from langchain_core.output_parsers import BaseOutputParser

# from Utils.PythonExecUtil import execute_python_code
from langchain_openai import ChatOpenAI
from langchain_experimental.utilities import PythonREPL

#### 自定义一个OutputParse

In [163]:
class PythonCodeParser(BaseOutputParser):
    """从OpenAI返回的文本中提取Python代码。"""

    def _remove_marked_lines(self, input_str: str) -> str:
        lines = input_str.strip().split('\n')
        if lines and lines[0].strip().startswith('```'):
            del lines[0]
        if lines and lines[-1].strip().startswith('```'):
            del lines[-1]

        ans = '\n'.join(lines)
        return ans

    def parse(self, text: str) -> str:
        # 使用正则表达式找到所有的Python代码块
        python_code_blocks = re.findall(r'```python\n(.*?)\n```', text, re.DOTALL)
        # 从re返回结果提取出Python代码文本
        python_code = None
        if len(python_code_blocks) > 0:
            python_code = python_code_blocks[0]
            python_code = self._remove_marked_lines(python_code)
        return python_code

#### 定义提示语模板

In [164]:
from langchain.prompts import PromptTemplate

excel_analyser_prompt = PromptTemplate.from_template("""
你的任务是先分析，再生成代码。

请根据用户的输入，一步步分析：
（1）用户的输入是否依赖某个条件，而这个条件没有明确赋值？
（2）我是否需要对某个变量的值做假设？

如果我需要对某个变量的值做假设，请直接输出：
```python
print("我需要知道____的值，才能生成代码。请完善你的查询。") # 请将____替换为需要假设的的条件
```

否则，生成一段Python代码，分析指定文件的内容。

你可以使用的库只包括：pandas, re, math, datetime, openpyxl
确保你的代码只使用上述库，否则你的代码将无法运行。

给定文件为：
{filename}

文件内容样例：
{inspections}

你输出的Python代码前后必须有markdown标识符，如下所示：
```python
# example code
print('hello world')
```

确保你的代码是可以运行的，文件名直接写死在代码里即可。
你生成代码中所有的常量都必须来自我给你的信息或来自文件本身。不要编造任何常量。
如果常量缺失，你的代码将无法运行。你可以拒绝生成代码，但是不要生成编造的代码。
确保你生成的代码最终以print的方式输出结果(回答用户的问题)。

用户输入：
{query}
""")

#### 定义执行链

In [165]:
llm = ChatOpenAI(
        model="gpt-4-1106-preview",
        temperature=0,
        model_kwargs={"seed": 42},
    )
analysis_chain = excel_analyser_prompt | llm | PythonCodeParser()

#### 生成 python 代码并执行

In [166]:
from langchain_experimental.utilities import PythonREPL
def excel_analyse(query, filename):
    """分析一个结构化文件（例如excel文件）的内容。"""

    filename = filename.strip()
    # columns = get_column_names(filename)
    inspections = get_first_n_rows(filename, 3)

    code = ""

    ## 打印详细信息
    color_print("\n#!/usr/bin/env python", CODE_COLOR, end="\n")

    for c in analysis_chain.stream({
        "query": query,
        "filename": filename,
        "inspections": inspections
    }):
        ## 打印详细信息
        color_print(c, CODE_COLOR, end="")
        ## 收集代码成果
        code += c

    if code:
        ans = PythonREPL().run(code)
        return ans
    else:
        return "没有找到可执行的Python代码"

#### 定义 tool 函数

In [167]:
# 输出工具
excel_analysis_tool = StructuredTool.from_function(
    func=make_safe_tool(excel_analyse),
    name="AnalyseExcel",
    description="""
        通过pandas数据处理脚本分析一个结构化文件（例如excel文件）的内容。
        输人中必须包含文件的完整路径和具体分析方式和分析依据，阈值常量等。
        如果输入信息不完整，你可以拒绝回答。
    """,
)

#### 使用示例

In [346]:
excel_analysis_tool.invoke({"query": "8月销售额", "filename":"./data/autogpt-demo/2023年8月-9月销售记录.xlsx"})


#!/usr/bin/env python
import pandas as pd

# 读取Excel文件
df = pd.read_excel('./data/autogpt-demo/2023年8月-9月销售记录.xlsx')

# 筛选出8月份的销售记录
df['销售日期'] = pd.to_datetime(df['销售日期'])  # 确保销售日期列是日期格式
august_sales = df[df['销售日期'].dt.month == 8]

# 计算8月份的销售额
august_sales['销售额'] = august_sales['单价(元)'] * august_sales['销售量']
total_august_sales = august_sales['销售额'].sum()

# 输出8月份的销售额
print(f"8月份的销售额为：{total_august_sales}元")

<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'8月份的销售额为：2605636元\n'

### 生成文档

#### 准备

In [168]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

#### 使用 LLM 生成文档

In [169]:
def write(query: str):
    """按用户要求生成文章"""
    template = ChatPromptTemplate.from_messages(
        [
            SystemMessagePromptTemplate.from_template("你是专业的文档写手。你根据客户的要求，写一份文档。输出中文。"),
            HumanMessagePromptTemplate.from_template("{query}"),
        ]
    )

    chain = {"query": RunnablePassthrough()} | template | ChatOpenAI() | StrOutputParser()

    return chain.invoke(query)

#### 定义 tool 函数

In [170]:
# 输出工具
document_generation_tool = StructuredTool.from_function(
    func=make_safe_tool(write),
    name="GenerateDocument",
    description="根据需求描述生成一篇正式文档",
)

#### 使用示例

In [350]:
# 示例
print(document_generation_tool.invoke("写一封邮件给张三，内容是：你好，我是李四。"))

亲爱的张三，

您好！我是李四，很高兴与您联系。

希望您一切都好。我想借此机会向您介绍一下我自己。我是一名专业的文档写手，拥有丰富的写作经验和技巧。我善于撰写各类文档，包括但不限于报告、提案、手册、说明书等。我以高质量的工作和准时交付而闻名。

我了解您对于一份文档的要求，我会确保根据您的要求和要求的时间表来完成工作。作为一名专业写手，我注重细节和准确性，同时也能够根据客户的需求进行灵活的调整。

如果您对我的服务感兴趣，并且希望了解更多详细信息，或者您有任何问题或特殊要求，请随时与我联系。我期待着与您合作，为您提供优质的文档写作服务。

谢谢您的时间和考虑。

祝您一切顺利！

李四


### 发送Email

#### 准备

In [35]:
import webbrowser
import urllib.parse
import re

#### 检查email格式是否合法

In [36]:
def _is_valid_email(email: str) -> bool:
    receivers = email.split(';')
    # 正则表达式匹配电子邮件
    pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    for receiver in receivers:
        if not bool(re.match(pattern, receiver.strip())):
            return False
    return True

#### 触发系统调用发送邮件

In [37]:
def send_email(
        to: str,
        subject: str,
        body: str,
        cc: str = None,
        bcc: str = None,
) -> str:
    """给指定的邮箱发送邮件"""

    if not _is_valid_email(to):
        return f"电子邮件地址 {to} 不合法"

    # 对邮件的主题和正文进行URL编码
    subject_code = urllib.parse.quote(subject)
    body_code = urllib.parse.quote(body)

    # 构造mailto链接
    mailto_url = f'mailto:{to}?subject={subject_code}&body={body_code}'
    if cc is not None:
        cc = urllib.parse.quote(cc)
        mailto_url += f'&cc={cc}'
    if bcc is not None:
        bcc = urllib.parse.quote(bcc)
        mailto_url += f'&bcc={bcc}'

    webbrowser.open(mailto_url)

    return f"状态: 成功\n备注: 已发送邮件给 {to}, 标题: {subject}"

#### 定义 tool 函数

In [38]:
# 发送邮件
email_tool = StructuredTool.from_function(
    func=make_safe_tool(send_email),
    name="SendEmail",
    description="给指定的邮箱发送邮件。确保邮箱地址是xxx@xxx.xxx的格式。多个邮箱地址以';'分割。",
)

#### 使用示例

In [370]:
send_email("43801@qq.com", "hello", "happy new year!")

'状态: 成功\n备注: 已发送邮件给 43801@qq.com, 标题: hello'

### FINISH

In [39]:
finish_placeholder = StructuredTool.from_function(
    func=lambda: None,
    name="FINISH",
    description="用于表示任务完成的占位符工具"
)

## toolkit 定义

In [171]:
# 自定义工具集
tools = [
    directory_inspection_tool,
    document_qa_tool,
    document_generation_tool,
    email_tool,
    excel_inspection_tool,
    excel_analysis_tool,
    finish_placeholder,
]

<div class="alert alert-info">
<b>工具集：</b><br/>
    以上就是所有的工具集定义了，可以在后面所有实践中共用。
</div>

## Prompt定义

<div class="alert alert-warning">
<b>构建Promp：</b><br/>
    主流程prompt较为复杂，可以结合局部模板或pipeline分层构建，以方便维护。<br>
（在王卓然老师的代码中有精彩的提炼）
</div>

下面是分布构建的过程，最后使用较简单的方式合并在一起。

### 预定义模板

#### 约束条件

In [41]:
constraints = """
1. 每次你的决策只使用一种工具，你可以使用任意多次。
2. 确保你调用的指令或使用的工具在下述给定的工具列表中。
3. 确保你的回答不会包含违法或有侵犯性的信息。
4. 如果你已经完成所有任务，确保以"FINISH"指令结束。
5. 用中文思考和输出。
6. 如果执行某个指令或工具失败，尝试改变参数或参数格式再次调用。
7. 你生成的回复必须遵循上文中给定的事实信息。不可以编造信息。DO NOT MAKE UP INFORMATION.
8. 如果得到的结果不正确，尝试更换表达方式。
9. 已经得到的信息，不要反复查询。
10. 确保你生成的动作是可以精确执行的。动作做中可以包括具体方法和目标输出。
11. 看到一个概念时尝试获取它的准确定义，并分析从哪些输入可以得到它的具体取值。
12. 生成一个自然语言查询时，请在查询中包含全部的已知信息。
13. 在执行分析或计算动作前，确保该分析或计算中涉及的所有子概念都已经得到了定义。
14. 你不可以打印一个文件的全部内容，这样的操作代价太大，且会造成不可预期的后果，是被严格禁止的。
15. 不要向用户提问。
"""

#### 限定资源

In [42]:
resources = """
1. 你可以查阅本地文件列表。
2. 你可以读取本地文件。
3. 你可以通过代码操作本地文件。
4. 你有非常优秀的逻辑分析能力，可以通过因果关系找到最优的解决方案。
"""

#### 表现评估

In [43]:
performance_evaluation = """
1. 尽你最大的努力，用你最好的水平，通过分析和检查，做出最好的决定。
2. 带着全局观，自我反思你计划与动作。
3. 考虑你之前的策略与决策来改善的你的计划。
4. 如果你反复得到相同的结果，修改你的计划和决策，避免死循环。
5. 如果你当前的动作无法获取到需要的信息，尝试展开关键概念的定义，再重新推理。
"""

#### 思考过程

In [44]:
thought_instructions = """
关键概念: 任务中涉及的组合型概念或实体。已经明确获得取值的关键概念，将其取值完整备注在概念后。
概念拆解: 将任务中的关键概念拆解为一系列待查询的子要素。每个关键概念一行，后接这个概念的子要素，每个子要素一行，行前以' -'开始。
反思:
    自我反思，观察以前的执行记录，思考概念拆解是否完整、准确。
    一步步思考是否每一个的关键概念或要素的查询都得到了准确的结果。
    反思你已经得到哪个要素/概念。你得到的要素/概念取值是否正确。从当前的信息中还不能得到哪些要素/概念。
    每个反思一行，行前以' -'开始。
思考: 观察执行记录和你的自我反思，并一步步思考
  （1）分析要素间的依赖关系，例如：
    i. 我是否需要先获得A的值/定义，才能通过A来获得B？
    ii. 如果我先获得A，是否可以通过A筛选B，减少穷举每个B的代价？
    iii. A和B是否存在在同一数据源中，我能否在获取A的同时获取B？
    iv. 是否还有更高效或更聪明的办法来查询一个概念或要素？
    v. 如果上一次尝试查询一个概念或要素时失败了，我是否可以尝试从另一个资源中再次查询？
    vi. 诸如此类，你可以扩展更多的思考 ...
  （2）根据以上分析，排列子要素间的查询优先级
  （3）找出当前需要获得取值的子要素
  注意，不要对要素的取值/定义做任何假设，确保你的信息来自给定的数据源！
推理: 根据你的反思与思考，一步步推理被选择的子要素取值的获取方式。如果前一次的计划失败了，请检查输入中是否包含每个概念/要素的明确定义，并尝试细化你的查询描述。
计划: 严格遵守以下规则，计划你的当前动作。
  （1）详细列出当前动作的执行计划。只计划一步的动作。PLAN ONE STEP ONLY!
  （2）一步步分析，包括数据源，对数据源的操作方式，对数据的分析方法。有哪些已知常量可以直接代入此次分析。
  （3）不要尝试计算文件的每一个元素，这种计算代价太高，是严格禁止的。你可以通过分析找到更有效的方法，比如条件筛选。
  （4）上述分析是否依赖某个要素的取值/定义，且该要素的取值/定义尚未获得。若果是，重新规划当前动作，确保所有依赖的要素的取值/定义都已经获得。
  （5）不要对要素的取值/定义做任何假设，确保你的信息来自给定的数据源。不要编造信息。DO NOT MAKE UP ANY INFORMATION!!!
  （6）确保你执行的动作涉及的所有要素都已获得确切的取值/定义。
  （7）如果全部子任务已完成，请用FINISH动作结束任务。
"""

### 需要动态生成的部份

#### 任务描述

`task_description`应当是智能体每次收到的任务目标，在调用智能体循环思考和行动前动态填入。

#### 工作目录

`work_dir`是智能体的工作文件夹，所有数据、文件都会放入这个位置。

In [45]:
work_dir = "./data/autogpt-demo"

#### 长时记忆

`long_term_memory`是智能体完成了一次任务之后的最终输出，应当由代码自动生成并作为变量动态填入。

#### 短时记忆

`short_term_memory`是智能体在经过一次思考和行动后产生的输出，应当由代码自动生成并作为变量动态填入。

#### 工具清单

`tools`是为智能体定义的Tool列表，应当由代码自动生成并作为变量动态填入。

#### 行动选择

`format_instructions`对LLM提出要求：按照预定义的 Pydantic 格式要求生成。

## 分步骤运行

### 定义Action输出解析

#### 定义Action

In [103]:
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any

class Action(BaseModel):
    name: str = Field(description="工具或指令名称")
    args: Optional[Dict[str,Any]] = Field(default=None, description="工具或指令参数，由参数名称和参数值组成")

#### OuptutParser

In [104]:
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser

# 解析Action
action_output_parser = PydanticOutputParser(pydantic_object=Action)

# 实现自动纠错
robust_parser = OutputFixingParser.from_llm(parser=action_output_parser, llm=llm)

#### 提示语中的action_parser

In [105]:
action_parser = _chinese_friendly(action_output_parser.get_format_instructions())

### main_prompt

In [106]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("""
你是强大的AI助手，可以使用工具与指令自动化解决问题。

## 你必须遵循以下约束来完成任务:
{constraints}

## 你的任务是:
{task_description}
如果此任务显示“无”、“没有了”、“已完成”或类似表达，你直接输出下述工具中的FINISH即可。

## 你需要的所有文件资料都在以下目录:
dir_path={work_dir}

## 你可以使用以下工具或指令，它们又称为动作或actions:
{tools}

## 你可以使用的资源包括:
{resources}

## 你需要评估你的表现:
{performance_evaluation}

## 相关的历史记录:
{long_term_memory}

## 当前的任务执行记录:
{short_term_memory}

## 输出形式：
###（1）首先，根据以下格式说明，输出你的思考过程:
{thought_instructions}

###（2）然后，根据以下格式说明，输出你选择执行的动作/工具:
{format_instructions}
""")

In [107]:
print(prompt.input_variables)

['constraints', 'format_instructions', 'long_term_memory', 'performance_evaluation', 'resources', 'short_term_memory', 'task_description', 'thought_instructions', 'tools', 'work_dir']


In [108]:
main_prompt = prompt.partial(
    constraints=constraints,
    work_dir="./data/autogpt-demo",
    resources=resources,
    performance_evaluation=performance_evaluation,
    thought_instructions=thought_instructions,
    format_instructions=action_parser
)

In [109]:
print(main_prompt.input_variables)

['long_term_memory', 'short_term_memory', 'task_description', 'tools']


### final_prompt

In [110]:
final_prompt = PromptTemplate.from_template("""
你的任务是:
{task_description}

经过以下的思考过程，你已经完成任务:
{short_term_memory}

现在请详细给出你的最终答案:
""")

### 确定任务

In [111]:
task = "9月份的销售额是多少"

### 执行准备

#### short_term_memory

In [112]:
from langchain.memory import ConversationTokenBufferMemory

In [113]:
# 构造一个基于Token缓存的记忆体
short_term_memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=4000,
)

# 初始化短时记忆
short_term_memory.save_context(
    {"input": "\n初始化"},
    {"output": "\n开始"}
)

#### reason_chain

In [114]:
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.tools.render import render_text_description

In [115]:
llm = ChatOpenAI(
    model="gpt-4-1106-preview",
    temperature=0,
    model_kwargs={
        "seed": 42
    },
)

<div class="alert alert-info">
<b>reason_chain：</b><br/>
此时，只剩下了short_term_memory、long_term_memory和tools需要填充：<br>
其中，short_term_memory用于保存中间步骤的运行结果，等待智能体运行时填充。<br>
另外两个参数则有助于应用的灵活配置。
</div>

In [116]:
reason_chain = main_prompt.partial(task_description=task)|llm

reason_chain.first.input_schema()

PromptInput(long_term_memory=None, short_term_memory=None, tools=None)

In [117]:
chain = main_prompt.partial(task_description="9月份销售额")|llm
resp = chain.stream({'tools':"", 'long_term_memory':"", 'short_term_memory':""})
for chunk in resp:
    print(chunk.content, end="")

### 思考过程

关键概念: 9月份销售额

概念拆解:
- 销售额: 通常指在一定时期内销售产品或服务所获得的收入总额。
  - 时间范围: 9月份
  - 数据格式: 数字，可能是一个总和或者是分条目累加的结果。

反思:
- 之前的执行记录中没有提供关于如何获取销售额的具体信息。
- 需要检查是否有包含销售数据的文件，特别是与9月份相关的。
- 目前还没有得到任何销售额的具体数值或计算方法。

思考:
- 需要先检查是否存在包含销售数据的文件。
- 如果存在多个文件，需要确定哪个文件包含了9月份的销售数据。
- 如果销售数据分散在多个文件中，可能需要汇总这些数据来计算9月份的销售额。
- 如果文件名或内容包含日期信息，可以通过日期筛选出9月份的数据。

推理:
- 首先，需要查看目录下的文件列表，找出可能包含销售数据的文件。
- 然后，读取这些文件，查找与9月份销售额相关的数据。
- 如果文件内容包含日期和销售额，可以通过筛选日期来获取9月份的销售数据。

计划:
- 执行动作，查看目录下的文件列表。
- 分析文件列表，找出可能包含9月份销售数据的文件。
- 如果找到相关文件，计划读取文件内容以获取销售数据。

### 执行动作

```json
{
  "name": "查阅本地文件列表",
  "args": {
    "dir_path": "./data/autogpt-demo"
  }
}
```

#### final_chain

In [118]:
final_chain = final_prompt.partial(task_description=task) | llm | StrOutputParser()

#### _step

In [119]:
from langchain.tools.render import render_text_description
from langchain.memory.chat_memory import BaseChatMemory

def _format_short_term_memory(memory: BaseChatMemory) -> str:
    messages = memory.chat_memory.messages
    string_messages = [messages[i].content for i in range(1,len(messages))]
    return "\n".join(string_messages)

# reason_chain
# short_term_memory
# tools
# action_output_parser
# _format_short_term_memory
# color_print
def _step(memory):
    # 输出LLM结果
    response = ""
    for chunk in reason_chain.stream({
        "short_term_memory": _format_short_term_memory(memory),
        "long_term_memory": "",
        "tools": render_text_description(tools)
    }):
        color_print(chunk.content, THOUGHT_COLOR, end="")
        response += chunk.content

    # 输出Action
    action = action_output_parser.parse(response)
    
    return action, response

#### _final_step

In [120]:
def _final_step(memory) -> str:
    """最后一步, 生成最终的输出"""
    response = final_chain.invoke({
        "short_term_memory": _format_short_term_memory(memory)
    })
    return response

#### _exec_action

In [121]:
from langchain.tools.base import BaseTool

def _find_tool(tool_name: str) -> Optional[BaseTool]:
    for tool in tools:
        if tool.name == tool_name:
            return tool
    return None

In [122]:
from pydantic import ValidationError

def _exec_action(action: Action) -> str:
    # 查找工具
    tool = _find_tool(action.name)
    # action_expr = format_action(action)
    if tool is None:
        observation = (
            f"Error: 找不到工具或指令 '{action.name}'. "
            f"请从提供的工具/指令列表中选择，请确保按对顶格式输出。"
        )

    else:
        try:
            # 执行工具
            observation = tool.run(action.args)
        except ValidationError as e:
            # 工具的入参异常
            observation = (
                f"Validation Error in args: {str(e)}, args: {action.args}"
            )
        except Exception as e:
            # 工具执行异常
            observation = f"Error: {str(e)}, {type(e).__name__}, args: {action.args}"

    return observation

### 尝试分步骤运行智能体

<div class="alert alert-info">
<b>一步步探索：</b><br/>
    万事具备，下面一步步探查智能体的每一个执行细节。
</div>

#### step1：思考

In [68]:
action, response = _step(short_term_memory)
print("\n\n - - - - - - ")
print(action)

### 思考过程

关键概念: 销售额 - 一个时间段内销售产品或服务所获得的总收入。

概念拆解:
- 销售额
  - 时间段: 需要确定是哪个月份的销售额，本任务中为9月份。
  - 数据来源: 需要确定销售额数据存储在哪里，可能是Excel表格、数据库或文档中。

反思:
- 我需要找到包含9月份销售额数据的文件。
- 我还没有得到任何要素/概念的取值。
- 当前信息中还不能确定销售额的具体数值。

思考:
- 我需要先确定数据存储的位置，才能查询到9月份的销售额。
- 如果销售数据存储在Excel表格中，我可以通过InspectExcel工具来查看表格结构，然后通过AnalyseExcel工具来获取具体的销售额。
- 如果销售数据存储在Word或PDF文档中，我可以使用AskDocument工具来查询。
- 我需要先使用ListDirectory工具来查看数据存储的位置。

推理:
- 根据任务要求，我需要查找包含9月份销售额数据的文件，这可能是一个Excel文件或其他格式的文档。
- 我将首先使用ListDirectory工具来列出可能包含销售数据的文件夹内容。

计划:
- 我将使用ListDirectory工具来列出"./data/autogpt-demo"目录下的文件和文件夹，以便找到可能包含9月份销售额数据的文件。

### 执行动作

```json
{
  "name": "ListDirectory",
  "args": {
    "path": "./data/autogpt-demo"
  }
}
```

 - - - - - - 
name='ListDirectory' args={'path': './data/autogpt-demo'}


#### step1：行动

In [67]:
observation = _exec_action(action)

In [68]:
color_print(f"\n----\n结果:\n{observation}", OBSERVATION_COLOR)


----
结果:
.DS_Store
2023年8月-9月销售记录.xlsx
供应商名录.xlsx
求职简历.docx
供应商资格要求.pdf


#### step1：观察

In [69]:
short_term_memory.save_context(
    {"input": response},
    {"output": "返回结果:\n" + observation}
)

#### step2：思考

In [70]:
action, response = _step(short_term_memory)

### 思考过程：

关键概念: 9月份的销售额

概念拆解:
- 销售额: 某个时间段内完成销售的总金额
- 9月份: 表示查询的特定时间段范围

反思:
- 文件列表中有一个名为“2023年8月-9月销售记录.xlsx”的文件，这个文件很可能包含了9月份的销售记录。
- 我需要检查这个Excel文件，以确定它是否包含9月份的销售数据。

思考:
- 我需要使用InspectExcel工具来查看“2023年8月-9月销售记录.xlsx”文件的内容和结构。
- 这将帮助我了解文件的列名和数据结构，以便我可以进一步分析9月份的销售额。

推理:
- 我将调用InspectExcel工具，查看“2023年8月-9月销售记录.xlsx”文件的前3行数据。

计划:
- 我将执行InspectExcel动作，参数为文件名“2023年8月-9月销售记录.xlsx”和默认行数3。

### 执行动作：

```json
{
  "name": "InspectExcel",
  "args": {
    "filename": "./data/autogpt-demo/2023年8月-9月销售记录.xlsx"
  }
}
```

#### step2：行动

In [71]:
print(action)

name='InspectExcel' args={'filename': './data/autogpt-demo/2023年8月-9月销售记录.xlsx'}


In [72]:
observation = _exec_action(action)
color_print(f"\n----\n结果:\n{observation}", OBSERVATION_COLOR)


----
结果:
这是 './data/autogpt-demo/2023年8月-9月销售记录.xlsx' 文件的工作表名称：

['2023年8月-9月销售记录']

这是 './data/autogpt-demo/2023年8月-9月销售记录.xlsx' 文件第一个工作表的列名：

品类
产品名
单价(元)
销售量
销售日期
供应商

这是 './data/autogpt-demo/2023年8月-9月销售记录.xlsx' 文件第一个工作表的前3行样例：

   品类                产品名  单价(元)  销售量       销售日期        供应商
   手机       Xiaomi Mi 11   4999   20 2023-08-02   北京科技有限公司
   耳机    Sony WH-1000XM4   2999   15 2023-08-03   上海音响有限公司
笔记本电脑 Lenovo ThinkPad X1   8999   10 2023-08-05 深圳创新科技有限公司


<div class="alert alert-warning">
<b>参数解析：</b><br/>
    如果想要参数解析这一步准确，就要提供较详细的工具函数参数指引，否则LLM会根据惯用的方法来推测参数名称。
</div>

#### step2：观察

In [73]:
short_term_memory.save_context(
    {"input": response},
    {"output": "返回结果:\n" + observation}
)

#### step3：思考

In [74]:
action, response = _step(short_term_memory)

### 思考过程：

关键概念: 9月份的销售额

概念拆解:
- 销售额: 某个时间段内的销售总金额
- 9月份: 表示查询的特定时间段范围

反思:
- 根据文件探查结果得知，“2023年8月-9月销售记录.xlsx”文件包含了销售日期这一列，这表示我可以用它来筛选9月份的数据。
- 为了计算9月份的销售额，需要对所有9月份的销售记录进行汇总。
- 我会需要列名中的“销售量”和“单价(元)”来计算销售额

思考:
- 我需要将销售量与单价相乘以得到每项销售的总额，然后对所有9月份的销售进行求和。

推理:
- 要进行这样的计算，必须使用AnalyseExcel工具。
- 需要提供完整的Excel文件路径、筛选9月份数据的条件以及需要执行的计算方式。

计划:
- 我将执行AnalyseExcel动作。
- 文件路径为：“./data/autogpt-demo/2023年8月-9月销售记录.xlsx”。
- 我将计算筛选出销售日期在2023年9月1日到2023年9月30日范围内的所有记录的销售总额。
- 为此，我需要进行条件筛选和数据聚合。

### 执行动作：

```json
{
  "name": "AnalyseExcel",
  "args": {
    "query": "SELECT SUM(`单价(元)` * `销售量`) FROM `2023年8月-9月销售记录` WHERE `销售日期` >= '2023-09-01' AND `销售日期` <= '2023-09-30'",
    "filename": "./data/autogpt-demo/2023年8月-9月销售记录.xlsx"
  }
}
```

#### step3：行动

In [75]:
observation = _exec_action(action)
color_print(f"\n----\n结果:\n{observation}", OBSERVATION_COLOR)


#!/usr/bin/env python
import pandas as pd

# 读取Excel文件
df = pd.read_excel('./data/autogpt-demo/2023年8月-9月销售记录.xlsx', sheet_name='2023年8月-9月销售记录')

# 确保销售日期为datetime类型
df['销售日期'] = pd.to_datetime(df['销售日期'])

# 设置日期范围
start_date = pd.Timestamp('2023-09-01')
end_date = pd.Timestamp('2023-09-30')

# 过滤出符合日期范围的数据
date_filtered_df = df[(df['销售日期'] >= start_date) & (df['销售日期'] <= end_date)]

# 计算总销售额
sales_sum = (date_filtered_df['单价(元)'] * date_filtered_df['销售量']).sum()

# 输出结果
print(f"在2023年9月的销售总额是：{sales_sum}元")

Python REPL can execute arbitrary code. Use with caution.



----
结果:
在2023年9月的销售总额是：2851099元



#### step3：观察

In [76]:
short_term_memory.save_context(
    {"input": response},
    {"output": "返回结果:\n" + observation}
)

#### step4：思考

In [77]:
action, response = _step(short_term_memory)

### 思考过程：

关键概念: 9月份的销售额 - 2851099元

概念拆解:
- 销售额: 某个时间段内完成销售的总金额
- 9月份: 表示查询的特定时间段范围

反思:
- 我已经使用AnalyseExcel工具计算出了9月份的销售总额。
- 结果显示9月份的销售总额为2851099元，这个结果是基于“2023年8月-9月销售记录.xlsx”文件中的数据计算得出的。
- 目前没有其他文件或数据源表明有关9月份销售额的信息，因此我可以认为这个结果是准确的。

思考:
- 我已经得到了9月份的销售额，这是本次任务的目标。
- 根据任务指示，完成所有任务后应该使用FINISH指令结束。

推理:
- 由于我已经得到了9月份的销售额，并且没有其他任务需要执行，我应该使用FINISH指令来结束任务。

计划:
- 我将执行FINISH动作来结束任务。

### 执行动作：

```json
{
  "name": "FINISH",
  "args": null
}
```

#### step4：结束

In [82]:
print(action)

name='FINISH' args=None


In [83]:
reply = ""
if action.name == "FINISH":
    color_print(f"\n----\nFINISH", OBSERVATION_COLOR)
    reply = _final_step(short_term_memory)


----
FINISH


In [84]:
print(reply)

完成任务:

9月份的销售额是 2,851,099 元。


## 在自定义循环中运行

### 实现循环框架

In [172]:
class AutoGPT:
    def run(self, task_description):
        print("等我询问GPT...")
        print("假装我经过复杂的计算过程...")
        print("FINISH")
        return "这是我的结论"

In [173]:
def launch_agent():
    human_icon = "\U0001F468"
    ai_icon = "\U0001F916"

    while True:
        task = input(f"{ai_icon}：有什么可以帮您？\n{human_icon}：")
        if task.strip().lower() == "quit":
            break
        agent = AutoGPT()
        reply = agent.run(task)
        print(f"{ai_icon}：{reply}\n")

In [174]:
agent = AutoGPT()

In [175]:
launch_agent()

🤖：有什么可以帮您？
👨： qu it


等我询问GPT...
假装我经过复杂的计算过程...
FINISH
🤖：这是我的结论



🤖：有什么可以帮您？
👨： quit


### 构建智能体

#### 执行链

In [176]:
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.tools.render import render_text_description

In [177]:
class AgentChain:
    action_parser = _chinese_friendly(action_output_parser.get_format_instructions())

    def __init__(self):
        self.llm = ChatOpenAI(
            model="gpt-4-1106-preview",
            temperature=0,
            model_kwargs={"seed": 42},
        )

    def get_reason_chain(self, task):
        prompt = main_prompt.partial(task_description=task, format_instructions=action_parser)
        reason_chain = prompt | self.llm | StrOutputParser()
        return reason_chain

    def get_final_chain(self, task):
        prompt = final_prompt.partial(task_description=task)
        final_chain = prompt | self.llm | StrOutputParser()
        return final_chain

#### 记忆体

In [178]:
from langchain.memory.chat_memory import BaseChatMemory
from langchain.memory import ConversationTokenBufferMemory

class AgentMemory:
    # AgentMemory所需要的 LLM 仅用于计算token，并不用于生成
    def __init__(self, llm = ChatOpenAI()):        
        # 构造一个基于Token缓存的记忆体
        self.memory = ConversationTokenBufferMemory(
            llm=llm,
            max_token_limit=4000,
        )
        
        # 初始化短时记忆
        self.memory.save_context(
            {"input": "\n初始化"},
            {"output": "\n开始"}
        )

    # 保存记忆
    def save(self, input, output):
        self.memory.save_context(input, output)

    # 提取记忆
    def load(self) -> str:
        messages = self.memory.chat_memory.messages
        string_messages = [messages[i].content for i in range(1,len(messages))]
        return "\n".join(string_messages)

#### _exec_action

In [179]:
from langchain.tools.base import BaseTool

def _find_tool(tool_name: str) -> Optional[BaseTool]:
    for tool in tools:
        if tool.name == tool_name:
            return tool
    return None

In [180]:
from pydantic import ValidationError

def _exec_action(action: Action) -> str:
    # 查找工具
    tool = _find_tool(action.name)
    # action_expr = format_action(action)
    if tool is None:
        observation = (
            f"Error: 找不到工具或指令 '{action.name}'. "
            f"请从提供的工具/指令列表中选择，请确保按对顶格式输出。"
        )

    else:
        try:
            # 执行工具
            observation = tool.run(action.args)
        except ValidationError as e:
            # 工具的入参异常
            observation = (
                f"Validation Error in args: {str(e)}, args: {action.args}"
            )
        except Exception as e:
            # 工具执行异常
            observation = f"Error: {str(e)}, {type(e).__name__}, args: {action.args}"

    return observation

#### Agent

In [181]:
class AutoGPT():
    def __init__(self):
        # 最大思考步骤数
        self.max_thought_steps = 10
        # 链
        self.chain = AgentChain()
        # 记忆体
        self.short_memory = AgentMemory()

    def _step(self, task):
        # 输出LLM结果
        response = ""
        chain = self.chain.get_reason_chain(task)
        for s in chain.stream({
            "short_term_memory": self.short_memory.load(),
            "long_term_memory": "",
            "tools": render_text_description(tools)
        }):
            color_print(s, THOUGHT_COLOR, end="")
            response += s
    
        # 输出Action
        action = action_output_parser.parse(response)
        
        return action, response

    def _final_step(self, task) -> str:
        """最后一步, 生成最终的输出"""
        chain = self.chain.get_final_chain(task)
        response = chain.invoke({
            "short_term_memory": self.short_memory.load()
        })
        return response
        
    def run(self, task_description):
        # 实际思考步骤数
        thought_step_count = 0

        # 结论
        reply = ""

        # 思考循环
        while thought_step_count < self.max_thought_steps:
            # 思考
            action, response = self._step(task_description)

            # 结束
            if action.name == "FINISH":
                # 如果返回的动作是 FINISH 就中断循环
                color_print(f"\n----\nFINISH", OBSERVATION_COLOR)
                reply = self._final_step(task_description)
                break
            
            else:
                # 否则执行动作中指定的工具
                # 提取工具执行结果，以供思考时观察
                observation = _exec_action(action)
                color_print(f"\n----\n结果:\n{observation}", OBSERVATION_COLOR)
                
                # 记录结果，提供观察
                self.short_memory.save(
                    {"input": response},
                    {"output": "返回结果:\n" + observation}
                )

                # 累加思考步骤数，继续思考
                thought_step_count += 1

        # 处理无法得出结论的情况
        if not reply:
            reply = "抱歉，我没能完成您的任务。"
            
        # 返回结论
        return reply

#### 运行

In [182]:
def launch_agent():
    human_icon = "\U0001F468"
    ai_icon = "\U0001F916"

    while True:
        task = input(f"{ai_icon}：有什么可以帮您？\n{human_icon}：")
        if task.strip().lower() == "quit":
            break
        agent = AutoGPT()
        reply = agent.run(task)
        print(f"{ai_icon}：{reply}\n")

In [183]:
launch_agent()

🤖：有什么可以帮您？
👨： 帮我找出销售额不达标的供应商


### 思考过程

关键概念: 销售额不达标的供应商 - 需要找出销售额低于某个阈值的供应商名单。

概念拆解:
- 销售额: 需要获取供应商的销售额数据。
- 达标阈值: 需要知道销售额的达标标准是多少。
- 供应商名单: 需要获取所有供应商的列表。

反思:
- 我需要先知道销售额的达标标准，才能判断哪些供应商没有达标。
- 我需要从数据源中获取供应商的销售额数据。
- 我还没有得到销售额的达标标准和供应商销售额的具体数据。
- 我需要检查是否有文件包含这些信息。

思考:
- 我需要先检查目录中有哪些文件，可能会包含供应商销售额和达标阈值的信息。
- 如果找到了包含销售额数据的文件，我可以使用InspectExcel工具来查看文件结构。
- 如果文件中包含达标阈值，我可以使用AnalyseExcel工具来分析不达标的供应商。

推理:
- 我将首先使用ListDirectory工具来查看目录中的文件列表，这是获取销售额数据和达标阈值的第一步。

计划:
- 使用ListDirectory工具来查看目录./data/autogpt-demo中的文件列表。

### 执行动作

```json
{
  "name": "ListDirectory",
  "args": {
    "path": "./data/autogpt-demo"
  }
}
```
----
结果:
.DS_Store
2023年8月-9月销售记录.xlsx
供应商名录.xlsx
求职简历.docx
供应商资格要求.pdf
### 思考过程

关键概念: 销售额不达标的供应商 - 需要找出销售额低于某个阈值的供应商名单。

概念拆解:
- 销售额: 需要获取供应商的销售额数据。
- 达标阈值: 需要知道销售额的达标标准是多少。
- 供应商名单: 需要获取所有供应商的列表。

反思:
- 我已经得到了文件列表，其中可能包含销售额数据的是"2023年8月-9月销售记录.xlsx"。
- 我还没有得到销售额的达标标准和供应商销售额的具体数据。
- 我需要检查"2023年8月-9月销售记录.xlsx"文件来获取销售额数据。
- 我还需要找到销售额的达标阈值，这可能在"供应商资格要求.pdf"文件中。

思考:
- 我需要先使用InspectExcel工具来查看"2023年8月-9月销售记录.x

🤖：有什么可以帮您？
👨： quit


## 在langchain智能体架构中运行

### 打印AgentExcutor的日志流

In [239]:
def agent_print(agent_steram_result):
    for chunk in agent_steram_result:
        if "actions" in chunk:
            print("\n\n-------------actions-----------------")
            for action in chunk["actions"]:
                for s in action.log:
                    print(s, end="", flush=True)
                print(f"调用工具：{action.tool} ，参数为：{action.tool_input}", flush=True)
        elif "steps" in chunk:
            print("\n\n-------------steps-----------------")
            for step in chunk["steps"]:
                print("\n\n获得结果:")
                for s in step.observation:
                    print(s, end="", flush=True)
        elif "log" in chunk:
            for s in chunk.log:
                print(s, end="", flush=True)
        elif "tool" in chunk:
            print(f"调用工具：{action.tool} ，参数为：{action.tool_input}", flush=True)
        else:
            print("\n\n-------------其他-----------------")
            print(chunk, flush=True)

### 输出解析

In [240]:
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any

class Action(BaseModel):
    name: str = Field(description="工具或指令名称")
    args: Optional[Dict[str,Any]] = Field(default=None, description="工具或指令参数，由参数名称和参数值组成")

In [263]:
from langchain.agents.agent import AgentOutputParser, AgentAction, AgentFinish
from typing import Union
from langchain.output_parsers import PydanticOutputParser

# 解析Action
action_output_parser = PydanticOutputParser(pydantic_object=Action)

class MyActionOutputParser(AgentOutputParser):
    """解析单个动作的智能体action和输入参数
    结合pydantic可精确捕捉输入参数。
    """

    def get_format_instructions(self) -> str:
        return "请预定义pydantic类型"

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        results = []
        # 提取json结构中的参数
        for action in action_output_parser.stream(text):
            # 按照AgentExcutor中的定义输出 AgentFinish 或 AgentAction
            print(".", end="", flush=True)
            if(action.name == "FINISH"):
                results.append(AgentFinish({"output": "完成任务"}, text))
            else:
                # 检查参数的值，如果为 None，设置一个默认值
                name = action.name if action.name is not None else "no_tool_name"
                args = action.args if action.args is not None else "no_args"
                text = text if text is not None else "empty text"
                results.append(AgentAction(name, args, text))
        return results
        # print(".", end="", flush=True)
        # # 提取json结构中的参数
        # action = action_output_parser.invoke(text)
    
        # # 按照AgentExcutor中的定义输出 AgentFinish 或 AgentAction
        # if(action.name == "FINISH"):
        #     return AgentFinish({"output": "完成任务"}, text)
        # else:
        #     # 检查参数的值，如果为 None，设置一个默认值
        #     name = action.name if action.name is not None else "no_tool_name"
        #     args = action.args if action.args is not None else "no_args"
        #     text = text if text is not None else "empty text"
        #     return AgentAction(name, args, text)    
    
    async def aparse(self, text: str) -> List[Union[AgentAction, AgentFinish]]:
        results = []
        # 提取json结构中的参数
        async for action in action_output_parser.astream(text):
            # 按照AgentExcutor中的定义输出 AgentFinish 或 AgentAction
            print("*", end="", flush=True)
            if(action.name == "FINISH"):
                results.append(AgentFinish({"output": "完成任务"}, text))
            else:
                # 检查参数的值，如果为 None，设置一个默认值
                name = action.name if action.name is not None else "no_tool_name"
                args = action.args if action.args is not None else "no_args"
                text = text if text is not None else "empty text"
                results.append(AgentAction(name, args, text))
        return results

    @property
    def _type(self) -> str:
        return "customize-single-input"

### 提示语

In [264]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("""
你是强大的AI助手，可以使用工具与指令自动化解决问题。

## 你必须遵循以下约束来完成任务:
{constraints}

## 你的任务是:
{task_description}
如果此任务显示“无”、“没有了”、“已完成”或类似表达，你直接输出下述工具中的FINISH即可。

## 你需要的所有文件资料都在以下目录:
dir_path={work_dir}

## 你可以使用以下工具或指令，它们又称为动作或actions:
{tools}

## 你可以使用的资源包括:
{resources}

## 你需要评估你的表现:
{performance_evaluation}

## 相关的历史记录:
{long_term_memory}

## 当前的任务执行记录:
{agent_scratchpad}

## 输出形式：
###（1）首先，根据以下格式说明，输出你的思考过程:
{thought_instructions}

###（2）然后，根据以下格式说明，输出你选择执行的动作/工具:
{format_instructions}
""")

In [265]:
main_prompt = prompt.partial(
    long_term_memory="",
    constraints=constraints,
    work_dir="./data/autogpt-demo",
    resources=resources,
    performance_evaluation=performance_evaluation,
    thought_instructions=thought_instructions,
    tools=render_text_description(tools),
    format_instructions=action_parser,
)

In [266]:
print(main_prompt.input_variables)

['agent_scratchpad', 'task_description']


### Agent

In [267]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description
from langchain.agents.format_scratchpad import format_log_to_str

In [268]:
llm = ChatOpenAI(
    model="gpt-4-1106-preview",
    temperature=0,
    model_kwargs={
        "seed": 42
    },
)

In [269]:
RunnablePassthrough.assign(
    task_description=lambda x: x["input"],
    agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"])
)

RunnableAssign(mapper={
  task_description: RunnableLambda(lambda x: x['input']),
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})

In [270]:
agent = (
    RunnablePassthrough.assign(
        task_description=lambda x: x["input"],
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]) # intermediate_steps是agent_executor要求的参数
    )
    | main_prompt
    | llm
    | MyActionOutputParser()
)

In [272]:
resp = agent.stream({"input": "9月份销售额", "intermediate_steps": ""})
agent_print(resp)

.

-------------其他-----------------
[AgentAction(tool='ListDirectory', tool_input={'path': './data/autogpt-demo'}, log='### 思考过程:\n\n关键概念: 9月份销售额 - 这是需要查询的具体数据。\n\n概念拆解:\n- 9月份销售额\n  - 销售数据文件：可能包含销售额的Excel或其他格式的文件。\n  - 时间范围：特定于2019年9月。\n  - 数据字段：需要找到表示销售额的具体字段。\n\n反思:\n- 我需要找到包含销售数据的文件，特别是包含2019年9月份数据的文件。\n- 我需要确认文件中是否有销售额的字段，并计算9月份的总销售额。\n- 我还没有得到任何要素/概念的取值。\n\n思考:\n- 我需要先找到包含销售数据的文件。\n- 一旦找到文件，我需要检查文件结构，确认是否包含销售额字段。\n- 如果文件包含所需数据，我将使用适当的工具来计算9月份的销售额。\n- 我需要确保使用的工具可以处理文件格式，并且能够执行所需的计算。\n\n推理:\n- 我将首先使用ListDirectory工具来查看目录中的文件列表，以找到可能包含销售数据的文件。\n- 一旦找到文件，我将使用InspectExcel工具来查看文件结构，确认是否有销售额字段。\n- 如果文件结构符合要求，我将使用AnalyseExcel工具来计算9月份的销售额。\n\n计划:\n- 使用ListDirectory工具查看目录`./data/autogpt-demo`中的文件列表。\n\n### 执行动作:\n\n```json\n{\n  "name": "ListDirectory",\n  "args": {\n    "path": "./data/autogpt-demo"\n  }\n}\n```')]


### 执行器

**注意：AgentExcuter需要单独做容错处理**

In [273]:
for tool in tools:
    tool.handle_tool_error = True

In [274]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

### 提问示例

- 9月份的销售额是多少
- 销售总额最大的产品是什么
- 帮我找出销售额不达标的供应商
- 给这两家供应商发一封邮件通知此事
- 对比8月和9月销售情况，写一份报告

### 运行：9月份销售额是多少

In [ ]:
agent_print(agent_executor.stream({"input": "9月份销售额是多少？"}))



> Entering new AgentExecutor chain...
.

-------------actions-----------------
### 思考过程:

关键概念: 9月份销售额 - 需要找到9月份的销售数据并计算总额。

概念拆解:
- 销售额: 通常指一定时期内的销售收入总和。
  - 销售数据: 包含日期和金额的数据。
  - 时间范围: 特定的月份，本任务中为9月份。
  - 数据来源: 可能是Excel表格或其他格式的文档。

反思:
- 我需要找到包含9月份销售数据的文件。
- 我需要确认文件格式，以便选择正确的工具进行数据提取。
- 我需要计算9月份的销售总额，这可能需要使用AnalyseExcel工具。

思考:
- 我需要先找到包含销售数据的文件，然后才能进行后续的分析。
- 如果销售数据在Excel文件中，我可以使用InspectExcel工具来查看文件结构，然后使用AnalyseExcel工具来计算销售总额。
- 如果销售数据在Word或PDF文档中，我需要使用AskDocument工具来查询特定的信息。

计划:
- 第一步，我将使用ListDirectory工具来查看目录下的文件列表，以找到可能包含9月份销售数据的文件。

### 执行动作/工具:

```json
{
  "name": "ListDirectory",
  "args": {
    "path": "./data/autogpt-demo"
  }
}
```调用工具：ListDirectory ，参数为：{'path': './data/autogpt-demo'}
### 思考过程:

关键概念: 9月份销售额 - 需要找到9月份的销售数据并计算总额。

概念拆解:
- 销售额: 通常指一定时期内的销售收入总和。
  - 销售数据: 包含日期和金额的数据。
  - 时间范围: 特定的月份，本任务中为9月份。
  - 数据来源: 可能是Excel表格或其他格式的文档。

反思:
- 我需要找到包含9月份销售数据的文件。
- 我需要确认文件格式，以便选择正确的工具进行数据提取。
- 我需要计算9月份的销售总额，这可能需要使用AnalyseExcel工具。

思考:
- 我需要先找到包含销售数据的文件，然后才能进行后续的分析。
- 如果销售

### 运行：销售总额最大的产品是什么

In [88]:
agent_print(agent_executor.stream({"input": "销售总额最大的产品是什么"}))



> Entering new AgentExecutor chain...
### 思考过程

关键概念: 销售总额最大的产品 - 需要找到销售数据中总额最高的产品名称。

概念拆解:
- 销售数据: 包含产品名称和销售总额的数据。
- 产品名称: 销售数据中的一个字段，代表产品。
- 销售总额: 销售数据中的一个字段，代表每个产品的销售金额累计。

反思:
- 我需要找到包含销售数据的文件，这可能是一个Excel文件。
- 我需要确认文件中是否有产品名称和销售总额这两个字段。
- 我需要分析销售数据，找到销售总额最大的产品名称。

思考:
- 我需要先使用ListDirectory动作来查看数据目录中有哪些文件。
- 如果目录中有Excel文件，我将使用InspectExcel动作来查看文件结构，确认是否包含产品名称和销售总额字段。
- 如果确认了字段存在，我将使用AnalyseExcel动作来分析数据，找出销售总额最大的产品。

推理:
- 首先，我需要查看目录中的文件列表，以确定数据文件的位置和类型。
- 然后，我需要检查数据文件的结构，以确认是否包含所需的字段。
- 最后，我需要分析数据，找出销售总额最大的产品。

计划:
- 第一步，我将使用ListDirectory动作来查看目录中的文件列表。

### 执行动作

```json
{
  "name": "ListDirectory",
  "args": {
    "path": "./data/autogpt-demo"
  }
}
```Calling Tool ```ListDirectory``` with input ```{'path': './data/autogpt-demo'}```
### 思考过程

关键概念: 销售总额最大的产品 - 需要找到销售数据中总额最高的产品名称。

概念拆解:
- 销售数据: 包含产品名称和销售总额的数据。
- 产品名称: 销售数据中的一个字段，代表产品。
- 销售总额: 销售数据中的一个字段，代表每个产品的销售金额累计。

反思:
- 我需要找到包含销售数据的文件，这可能是一个Excel文件。
- 我需要确认文件中是否有产品名称和销售总额这两个字段。
- 我需要分析销售数据，找到销售总额最大的产品名称。

思考:
- 我需要先使用ListDir

### 运行：帮我找出销售额不达标的供应商

In [89]:
agent_print(agent_executor.stream({"input": "帮我找出销售额不达标的供应商"}))



> Entering new AgentExecutor chain...
### 思考过程

关键概念: 销售额不达标的供应商 - 需要找出销售额低于某个阈值的供应商名单。

概念拆解:
- 销售额不达标
  - 需要知道达标的销售额是多少（阈值）
  - 需要知道每个供应商的销售额
- 供应商
  - 需要知道有哪些供应商
  - 需要知道供应商的销售额数据

反思:
- 我需要先知道销售额的达标阈值是多少。
- 我需要找到包含供应商销售额数据的文件。
- 我需要分析这个文件，找出销售额不达标的供应商。
- 我还没有得到任何要素/概念的取值。

思考:
- 我需要先找到包含供应商销售额数据的文件。
- 通过文件内容，我可以了解到达标的销售额阈值以及每个供应商的销售额。
- 如果销售额的达标阈值在文件中没有直接给出，我可能需要通过其他方式获取，比如文件中可能会有达标标准的描述。
- 我需要分析文件内容，找出销售额不达标的供应商。

推理:
- 我将使用ListDirectory工具来查看目录中有哪些文件。
- 如果目录中有Excel文件，我将使用InspectExcel工具来查看文件结构和内容。
- 如果目录中有Word或PDF文件，我可能需要使用AskDocument工具来查询销售额的达标阈值。

计划:
- 我将首先使用ListDirectory工具来查看目录中的文件列表。
- 如果找到了Excel文件，我将使用InspectExcel工具来查看文件的列名和前几行数据。
- 如果找到了Word或PDF文件，我可能需要使用AskDocument工具来查询销售额的达标阈值。

### 执行动作

```json
{
  "name": "ListDirectory",
  "args": {
    "path": "./data/autogpt-demo"
  }
}
```Calling Tool ```ListDirectory``` with input ```{'path': './data/autogpt-demo'}```
### 思考过程

关键概念: 销售额不达标的供应商 - 需要找出销售额低于某个阈值的供应商名单。

概念拆解:
- 销售额不达标
  - 需要知道达标的销售额是多少（阈值）
  - 需要知道每个供应商的销售额
- 供应商
 

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


{'query': '销售额的达标阈值是多少？(请用中文回答)', 'result': ' 月销售额达标阈值为人民币3万元。'}

ValidationError: 2 validation errors for HumanMessage
content
  str type expected (type=type_error.str)
content
  value is not a valid list (type=type_error.list)